In [ ]:
from autumn.core.inputs.database import get_input_db
import pycountry
import os
import pandas as pd
from autumn.settings.folders import DATA_PATH
from datetime import datetime

In [ ]:

input_db = get_input_db()

In [ ]:
manual_map = {
    "Democratic Republic of the Congo": "COD",
    "Laos": "LAO",
    "U.S. Virgin Islands": "VIR",
    "Bonaire": "BOS",
    "Curacao": "CUW",
    "Iran": "IRN",
    "Reunion": "REU",
    "Sint Maarten": "MAF",
    "Taiwan": "TWN",
    "USA": "USA"
}

def get_iso3(country):
    if country in manual_map:
        return manual_map[country]


    country_objects = pycountry.countries.search_fuzzy(country)
    if len(country_objects) == 1:
        return country_objects[0].alpha_3
    else:
        print(country)
        return pycountry.countries.get(name=country).alpha_3

### Excluded countries

In [ ]:
excluded_countries = ["Canary Islands", "Crimea"]

# Population size by age (n=201)

In [ ]:
pop = input_db.query(
    table_name='population', 
    conditions= {"year": 2020, "region": None}
)
pop_iso3s = pop.iso3.unique()
print(len(pop_iso3s))

## GISAID (n=204)

In [ ]:

gisaid_countries = input_db.query(
    table_name='gisaid', 
    # conditions= {"iso_code": "AUS"}
).Country.unique()
gisaid_countries = [c for c in gisaid_countries if c not in excluded_countries]
gisaid_iso3s = [get_iso3(c) for c in gisaid_countries]

gisaid_iso3s = list(dict.fromkeys(gisaid_iso3s)) # remove duplicates


In [ ]:
len(gisaid_iso3s)

## UNESCO (N=210)

In [ ]:
input_db.table_names()

In [ ]:
unesco_iso3s = list(input_db.query(
     table_name='school_closure', 
    # conditions= {"iso_code": "AUS"}
).country_id.unique())


In [ ]:
len(unesco_iso3s)

## SeroTracker
Using both national and sub-national estimates but tagging those that are national

In [ ]:
url="https://raw.githubusercontent.com/serotracker/sars-cov-2-data/main/serotracker_dataset.csv"  # current as at 27 Mar 2023
sero_data=pd.read_csv(url)

In [ ]:
sero_data.end_date = pd.to_datetime(sero_data.sampling_end_date, format="%Y/%m/%d")

In [ ]:
unity_filter = sero_data["is_unity_aligned"] == "Unity-Aligned"
national_filter = sero_data["estimate_grade"] == "National"
size_filter = sero_data["denominator_value"] >= 599
date_filter = sero_data.end_date <= datetime(2021, 5, 1)

In [ ]:
filtered_sero_data = sero_data[unity_filter & size_filter & date_filter]
filtered_sero_data_national = sero_data[unity_filter & size_filter & date_filter & national_filter]

In [ ]:
SeroTracker_iso3s = list(sero_data['alpha_3_code'].unique())
SeroTracker_iso3s_national = list(filtered_sero_data_national['alpha_3_code'].unique())

# Intercept

In [ ]:
included_iso3s = [iso3 for iso3 in gisaid_iso3s if iso3 in pop_iso3s and iso3 in unesco_iso3s and iso3 in SeroTracker_iso3s]
included_iso3s_national = [iso3 for iso3 in gisaid_iso3s if  iso3 in pop_iso3s and iso3 in unesco_iso3s and iso3 in SeroTracker_iso3s_national]
print(len(included_iso3s))
print(len(included_iso3s_national))

## Make a map

### All countries, including sub-national sero-surveys

In [ ]:
import plotly.express as px

print(f"N counries included: {len(included_iso3s)}")

df = pd.DataFrame({'country':included_iso3s, 'value': [1.]*len(included_iso3s)})
fig = px.choropleth(df, locations="country",
                    color="value", # lifeExp is a column of gapminder
                    # hover_name="country", # column to add to hover information
) #color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

### Only including national sero-surveys

In [ ]:
import plotly.express as px
print(f"N counries included: {len(included_iso3s_national)}")
df = pd.DataFrame({'country':included_iso3s_national, 'value': [1.]*len(included_iso3s_national)})
fig = px.choropleth(df, locations="country",
                    color="value", # lifeExp is a column of gapminder
                    # hover_name="country", # column to add to hover information
) #color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

### Dump list of included countries

In [ ]:
included_countries = [pycountry.countries.get(alpha_3=iso3).name for iso3 in included_iso3s]
included_countries_national = [pycountry.countries.get(alpha_3=iso3).name for iso3 in included_iso3s_national]

In [ ]:
included_dict = {
    "all": {iso3: country_name for (iso3, country_name) in zip(included_iso3s, included_countries)},
    "national":  {iso3: country_name for (iso3, country_name) in zip(included_iso3s_national, included_countries_national)}
}

In [ ]:
import yaml

with open('included_countries.yml', 'w') as outfile:
    yaml.dump(included_dict, outfile, default_flow_style=False)